# GA-1 (Non-tiered) – Colab Notebook

> This notebook mirrors the original GA-1 code with minimal/no logic changes.
> If your data is on Google Drive, mount Drive and adjust the file path.


In [ ]:
!pip install adjustText
!pip install folium
!pip install pillow
# RIG SCHEDULING OPTIMIZATION WITH GENETIC ALGORITHM
# Objective: Optimize rig scheduling using GA considering distance/time and BOPD

# LIBRARY
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.collections import LineCollection
from adjustText import adjust_text

#Tambahan
from datetime import timedelta, datetime # Jika kamu menghitung waktu tempuh dan tanggal eksekusi.
from sklearn.preprocessing import MinMaxScaler # Jika kamu melakukan normalisasi sebelum GA.
from itertools import permutations # Berguna untuk brute force (pembanding non-GA).
# Load Data if use Google Drive
# DATASET_PATH = "/content/drive/MyDrive/THESIS/DATASET_THESIS/Variant-3_RIG-6_RS_Beka_2024-2025.xlsx"
# df = pd.read_excel(DATASET_PATH)

df = pd.read_excel("Variant-4_RIG-24_WOWS_Mina_2022-2024.xlsx")
df.head()

print("Kolom:", df.columns)
print("Jumlah data:", df.shape)
print("Jumlah missing value:\n", df.isnull().sum())
print("Duplikat:", df.duplicated().sum())

# Convert DATE to datetime format
df['REQ_DT'] = pd.to_datetime(df['REQ_DT'])
df['START_DATETIME_JOB'] = pd.to_datetime(df['START_DATETIME_JOB'])
df['END_DATETIME_JOB'] = pd.to_datetime(df['END_DATETIME_JOB'])
df['Month'] = df['START_DATETIME_JOB'].dt.to_period('M')  # Periode bulanan
df.info()

df.isnull().sum()

# EDA

df.describe()

# Print the first and last start dates
first_start = df['START_DATE_JOB'].min()
last_start = df['START_DATE_JOB'].max()

print(f"First start date: {first_start}")
print(f"Last start date: {last_start}")

assert all(df['END_DATETIME_JOB'] >= df['START_DATETIME_JOB']) #Ada job dengan end date < start date

# Menampilkan urutan rute secara horizontal
print(" → ".join(df['WELL_ALIAS'].tolist()))

# Hapus dulu kolom yang akan dibuat ulang (jika sudah ada)
for col in ['Month_Label', 'Month_Sort', 'Year']:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

# Buat kolom baru
df['Month_Label'] = df['START_DATETIME_JOB'].dt.strftime('%B %Y')
df['Month_Sort'] = df['START_DATETIME_JOB'].dt.to_period('M').astype(str)
df['Year'] = df['START_DATETIME_JOB'].dt.year

# Group per bulan
month_counts = df.groupby(['Year', 'Month_Sort', 'Month_Label']).size().reset_index(name='Jumlah')
month_counts = month_counts.sort_values(by=['Year', 'Month_Sort'])

# Plot per tahun dengan garis rata-rata per tahun
years = month_counts['Year'].unique()
for year in years:
    data_year = month_counts[month_counts['Year'] == year]

    avg_jumlah = data_year['Jumlah'].mean()

    plt.figure(figsize=(10,5))
    plt.bar(data_year['Month_Label'], data_year['Jumlah'], color='green')
    plt.axhline(y=avg_jumlah, color='red', linestyle='--', linewidth=2, label=f'Rata-rata {year}: {avg_jumlah:.1f} jobs/month')

    plt.title(f'BASELINE - Aktual Execution Rate per Bulan Tahun {year}')
    plt.xlabel('Bulan')
    plt.ylabel('Jumlah Eksekusi Job')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Hapus kolom jika sudah ada
for col in ['Month_Label', 'Month_Sort', 'Year']:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

# Buat kolom baru
df['Month_Label'] = df['START_DATETIME_JOB'].dt.strftime('%b %Y')  # pakai singkatan bulan biar gak terlalu panjang
df['Month_Sort'] = df['START_DATETIME_JOB'].dt.to_period('M').astype(str)
df['Year'] = df['START_DATETIME_JOB'].dt.year

# Group data per bulan
month_counts = df.groupby(['Year', 'Month_Sort', 'Month_Label']).size().reset_index(name='Jumlah')
month_counts = month_counts.sort_values(by=['Month_Sort'])

# Buat list bulan unik sebagai sumbu X (urut berdasarkan waktu)
all_months = sorted(month_counts['Month_Sort'].unique())
month_labels = month_counts.drop_duplicates('Month_Sort').set_index('Month_Sort').loc[all_months]['Month_Label'].tolist()

plt.figure(figsize=(14,7))

# Plot bar untuk tiap bulan (total jumlah jobs di semua tahun)
# Jika data dari tahun berbeda di bulan sama ingin dijumlahkan, bisa group ulang di sini:
total_per_month = month_counts.groupby('Month_Sort')['Jumlah'].sum().reindex(all_months).fillna(0)
plt.bar(range(len(all_months)), total_per_month, color='green', label='Jumlah Eksekusi Job')

# Plot garis rata-rata per tahun (dotline)
years = month_counts['Year'].unique()
colors = plt.cm.get_cmap('tab10', len(years))  # buat warna berbeda tiap tahun

for i, year in enumerate(years):
    data_year = month_counts[month_counts['Year'] == year]
    avg_jumlah = data_year['Jumlah'].mean()
    plt.axhline(y=avg_jumlah, color=colors(i), linestyle='--', linewidth=2, label=f'Rata-rata {year}: {avg_jumlah:.1f} jobs/month')

# Set labels dan styling
plt.title('BASELINE - Aktual Execution Rate per Bulan (Gabungan Tahun)')
plt.xlabel('Tanggak Eksekusi Job')
plt.ylabel('Jumlah Eksekusi Job')
plt.xticks(ticks=range(len(all_months)), labels=month_labels, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# Visualisasi Waktu Permintaan (REQ_DT)
df['START_DATE_JOB'] = pd.to_datetime(df['START_DATE_JOB'])
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='START_DATE_JOB', y='BOPD', hue='WELL_ALIAS', palette='Set1', s=100)
plt.title('Scatter Plot of Executed Jobs by Production Output Over Time')
plt.xlabel('Execution Date')
plt.ylabel('BOPD')
plt.xticks(rotation=45)
plt.legend().remove()
plt.grid(True)
plt.show()

df_valid = df[(df['BOPD'] > 0) &
              df['SURFACE_LATITUDE'].notnull() &
              df['SURFACE_LONGITUDE'].notnull()].copy()

df_valid = df_valid.reset_index(drop=True)

# Hitung quantile dinamis
q1 = df_valid['BOPD'].quantile(0.25)
q2 = df_valid['BOPD'].quantile(0.50)
q3 = df_valid['BOPD'].quantile(0.75)

print(f"Q1 (25%): {q1:.2f}, Q2 (Median): {q2:.2f}, Q3 (75%): {q3:.2f}")

# Plot histogram dengan garis vertikal dari quantile
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(df_valid['BOPD'], bins=15, color='skyblue', edgecolor='black')

# Garis batas tier otomatis
plt.axvline(q1, color='blue', linestyle='--', label=f'Tier-3 (Q1) (≤ {q1:.1f})')
plt.axvline(q2, color='green', linestyle='--', label=f'Tier-2 (Median) ({q2:.1f})')
plt.axvline(q3, color='red', linestyle='--', label=f'Tier-1 (Q3) (≥ {q3:.1f})')

plt.title("Histogram of BOPD Distribution with Tier Thresholds")
plt.xlabel("BOPD")
plt.ylabel("Number of Wells")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius bumi dalam kilometer
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi/2.0)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c  # jarak dalam km

def create_distance_matrix(locations):
    n = len(locations)
    matrix = np.zeros((n, n))
    for i in range(n):
        lat1, lon1 = locations[i]
        for j in range(n):
            if i != j:
                lat2, lon2 = locations[j]
                matrix[i][j] = haversine(lat1, lon1, lat2, lon2)
    return matrix  # dalam km

# Hapus baris dengan data yang tidak lengkap
df_clean = df.dropna(subset=['SURFACE_LATITUDE', 'SURFACE_LONGITUDE', 'WELL_ALIAS'])

# Siapkan data bersih
locations = list(zip(df_clean['SURFACE_LATITUDE'], df_clean['SURFACE_LONGITUDE']))
well_ids = df_clean['WELL_ALIAS'].values
distance_matrix = create_distance_matrix(locations)
df_matrix = pd.DataFrame(distance_matrix, index=well_ids,
                         columns=well_ids)
# Tampilkan 5×5 sub‐tabel pertama dengan pembulatan 2 desimal
print(df_matrix.round(2).iloc[:6, :6])

def calculate_objective(route, matrix, bopd_list, duration_days):
    total_score = 0
    current_time = 0
    for i in range(len(route)):
        well_idx = route[i]
        if i == 0:
            travel_time = 0
        else:
            travel_time = matrix[route[i - 1]][well_idx] / 20

        job_time = duration_days[well_idx]
        if job_time < 0:
            job_time = 0
        total_time = travel_time + job_time
        production = bopd_list[well_idx]
        current_time += total_time
        score = total_time / production if production > 0 else float('inf')
        total_score += score
    return total_score

def create_route(n):
    route = list(range(1, n))
    random.shuffle(route)
    return [0] + route

#Testing
n = 10
route = create_route(n)
print(route)
print("Apakah dimulai dari 0?", route[0] == 0)
print("Apakah semua sumur unik?", len(set(route)) == n)

def crossover(p1, p2):
    size = len(p1)
    start, end = sorted(random.sample(range(1, size), 2))
    child = [-1] * size
    child[start:end] = p1[start:end]
    fill = [item for item in p2 if item not in child and item != 0]
    pointer = 0
    for i in range(1, size):
        if child[i] == -1:
            child[i] = fill[pointer]
            pointer += 1
    child[0] = 0
    return child

def mutate(route, rate=0.02):
    for i in range(1, len(route)):
        if random.random() < rate:
            j = random.randint(1, len(route) - 1)
            route[i], route[j] = route[j], route[i]
    return route

def genetic_algorithm(matrix, bopd_list, duration_days, generations=500, pop_size=400):
    best_scores = []
    population = [create_route(len(matrix)) for _ in range(pop_size)]
    for gen in range(generations):
        population.sort(key=lambda x: calculate_objective(x, matrix, bopd_list, duration_days))
        best_scores.append(calculate_objective(population[0], matrix, bopd_list, duration_days))
        next_gen = population[:5]
        while len(next_gen) < pop_size:
            parents = random.sample(population[:20], 2)
            child = mutate(crossover(parents[0], parents[1]))
            next_gen.append(child)
        population = next_gen
    best_route = min(population, key=lambda x: calculate_objective(x, matrix, bopd_list, duration_days))
    best_score = calculate_objective(best_route, matrix, bopd_list, duration_days)
    return best_route, best_score, best_scores

# Input untuk GA
df_valid = df[(df['BOPD'] > 0) & df['SURFACE_LATITUDE'].notnull() & df['SURFACE_LONGITUDE'].notnull()].copy()

duplicate_mask = df_valid.duplicated(subset=['SURFACE_LATITUDE', 'SURFACE_LONGITUDE'], keep=False)
df_valid.loc[duplicate_mask, 'SURFACE_LATITUDE'] += np.random.uniform(-0.0001, 0.0001, size=duplicate_mask.sum())
df_valid.loc[duplicate_mask, 'SURFACE_LONGITUDE'] += np.random.uniform(-0.0001, 0.0001, size=duplicate_mask.sum())

df_valid['duration_days'] = (
    (df_valid['END_DATETIME_JOB'] - df_valid['START_DATETIME_JOB'] - pd.to_timedelta(df_valid['MOVING_TIME'], unit='D'))
    .dt.total_seconds() / 86400
).fillna(2)

bopd_list = df_valid['BOPD'].tolist()
duration_days = df_valid['duration_days'].tolist()
locations = list(zip(df_valid['SURFACE_LATITUDE'], df_valid['SURFACE_LONGITUDE']))
duration_days = df_valid['duration_days'].tolist()

distance_matrix = create_distance_matrix(locations)

best_route, best_score, best_scores = genetic_algorithm(distance_matrix, bopd_list, duration_days)

print(f'Best route score: {best_score:.2f}')
print("Best Route (Index):", best_route)
print("Well Order:", df_valid.iloc[best_route]['WELL_ALIAS'].tolist())
total_bopd = sum([bopd_list[i] for i in best_route])
print(f'Total BOPD from best route: {total_bopd:.1f}')

import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.plot(best_scores, marker='o')
plt.title('Convergence (Approach GA-1)')
plt.xlabel('Generation')
plt.ylabel('Best Fitness Score')
plt.grid(True)

plt.text(0, best_scores[0], f'{best_scores[0]:.2f}',
         ha='left', va='bottom', fontsize=9, color='green')

plt.text(len(best_scores)-1, best_scores[-1], f'{best_scores[-1]:.2f}',
         ha='right', va='bottom', fontsize=9, color='red')

plt.tight_layout()
plt.show()

# Evaluasi rute hasil GA
t0 = pd.to_datetime(df_valid['START_DATETIME_JOB'].min())

df_ga = df_valid.iloc[best_route].reset_index(drop=True)

speed_km_per_day = 20

travel_times = [0]
for i in range(1, len(best_route)):
    prev = best_route[i - 1]
    curr = best_route[i]
    distance = distance_matrix[prev][curr]
    travel_time = distance / speed_km_per_day
    travel_times.append(travel_time)

df_ga['travel_time'] = travel_times
df_ga['total_time'] = df_ga['duration_days'] + df_ga['travel_time']
df_ga['cum_time'] = df_ga['total_time'].cumsum()

df_ga['executed_date'] = t0 + pd.to_timedelta(df_ga['cum_time'], unit='D')
df_ga['executed_month'] = df_ga['executed_date'].dt.to_period('M')
df_ga['executed_year'] = df_ga['executed_date'].dt.year

print("Urutan rute hasil GA:")
print(" → ".join(df_ga['WELL_ALIAS'].tolist()))
print("Travel time max:", df_ga['travel_time'].max(), "days")
print("Total waktu eksekusi:", df_ga['cum_time'].iloc[-1], "days")

df['Month_Label'] = df['START_DATETIME_JOB'].dt.strftime('%B %Y')
df['Month_Sort'] = df['START_DATETIME_JOB'].dt.to_period('M').astype(str)

baseline_jobs = df.groupby(['Month_Sort', 'Month_Label']).size().reset_index(name='Baseline Jobs')
baseline_prod = df.groupby(['Month_Sort', 'Month_Label'])['BOPD'].sum().reset_index(name='Baseline Production')

df_ga['Month_Label'] = df_ga['executed_date'].dt.strftime('%B %Y')
df_ga['Month_Sort'] = df_ga['executed_date'].dt.to_period('M').astype(str)
ga_jobs = df_ga.groupby(['Month_Sort', 'Month_Label']).size().reset_index(name='GA Jobs')
ga_prod = df_ga.groupby(['Month_Sort', 'Month_Label'])['BOPD'].sum().reset_index(name='GA Production')

years = sorted(set(df['START_DATETIME_JOB'].dt.year.unique()) | set(df_ga['executed_date'].dt.year.unique()))

for year in years:
    baseline = df[df['START_DATETIME_JOB'].dt.year == year].copy()
    ga_year = df_ga[df_ga['executed_date'].dt.year == year].copy()

    baseline['Month_Label'] = baseline['START_DATETIME_JOB'].dt.strftime('%B')
    baseline['Month_Sort'] = baseline['START_DATETIME_JOB'].dt.to_period('M').astype(str)
    ga_year['Month_Label'] = ga_year['executed_date'].dt.strftime('%B')
    ga_year['Month_Sort'] = ga_year['executed_date'].dt.to_period('M').astype(str)

    baseline_jobs = baseline.groupby(['Month_Sort', 'Month_Label']).size().reset_index(name='Baseline Jobs')
    ga_jobs = ga_year.groupby(['Month_Sort', 'Month_Label']).size().reset_index(name='GA Jobs')
    jobs_compare = pd.merge(baseline_jobs, ga_jobs, on=['Month_Sort', 'Month_Label'], how='outer').fillna(0).sort_values('Month_Sort')

    avg_jobs_baseline = jobs_compare['Baseline Jobs'].mean()
    avg_jobs_ga = jobs_compare['GA Jobs'].mean()

    bar_width = 0.4
    x = np.arange(len(jobs_compare))
    plt.figure(figsize=(14,6))
    total_jobs_baseline = jobs_compare['Baseline Jobs'].sum()
    total_jobs_ga = jobs_compare['GA Jobs'].sum()

    plt.bar(x - bar_width/2, jobs_compare['Baseline Jobs'], width=bar_width, color='steelblue',
            label=f'Baseline ({int(total_jobs_baseline)} jobs)')
    plt.bar(x + bar_width/2, jobs_compare['GA Jobs'], width=bar_width, color='purple',
            label=f'Genetic Algorithm ({int(total_jobs_ga)} jobs)')

    plt.axhline(y=avg_jobs_baseline, color='blue', linestyle='--', linewidth=2, label=f'Avg Baseline : {avg_jobs_baseline:.1f}')
    plt.axhline(y=avg_jobs_ga, color='darkmagenta', linestyle='--', linewidth=2, label=f'Avg GA : {avg_jobs_ga:.1f}')

    plt.title(f'Execution Rate per Month: Baseline vs Genetic Algorithm (GA-1)')
    plt.xlabel('Execution Date')
    plt.ylabel('Total Executed Job')
    plt.xticks(x, jobs_compare['Month_Label'], rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

    baseline_prod = baseline.groupby(['Month_Sort', 'Month_Label'])['BOPD'].sum().reset_index(name='Baseline Production')
    ga_prod = ga_year.groupby(['Month_Sort', 'Month_Label'])['BOPD'].sum().reset_index(name='GA Production')
    prod_compare = pd.merge(baseline_prod, ga_prod, on=['Month_Sort', 'Month_Label'], how='outer').fillna(0).sort_values('Month_Sort')

    avg_prod_baseline = prod_compare['Baseline Production'].mean()
    avg_prod_ga = prod_compare['GA Production'].mean()

    plt.figure(figsize=(14,6))
    total_prod_baseline = prod_compare['Baseline Production'].sum()
    total_prod_ga = prod_compare['GA Production'].sum()

    plt.plot(prod_compare['Month_Label'], prod_compare['Baseline Production'], marker='o', color='steelblue',
            label=f'Baseline ({int(total_prod_baseline)} BOPD)')
    plt.plot(prod_compare['Month_Label'], prod_compare['GA Production'], marker='o', color='purple',
            label=f'Genetic Algorithm ({int(total_prod_ga)} BOPD)')

    plt.axhline(y=avg_prod_baseline, color='blue', linestyle='--', linewidth=2, label=f'Avg Baseline Prod : {avg_prod_baseline:.1f}')
    plt.axhline(y=avg_prod_ga, color='darkmagenta', linestyle='--', linewidth=2, label=f'Avg GA Prod : {avg_prod_ga:.1f}')

    plt.title(f'Total Oil Recovery per Month: Baseline vs Genetic Algorithm (GA-1)')
    plt.xlabel('Execution Date')
    plt.ylabel('Total Oil Recovery (BOPD)')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()
